In [231]:
import pypyodbc as odbc
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder


In [208]:
DRIVER_NAME = 'SQL Server'
SERVER_NAME = 'Haitham'
DATABASE_NAME = 'ITIExaminationSystem'

CONNECTION_STRING = f'DRIVER={{{DRIVER_NAME}}};SERVER={SERVER_NAME};DATABASE={DATABASE_NAME};Trusted_Connection=yes;'

In [209]:
conn = odbc.connect(CONNECTION_STRING) 
print(conn)

In [210]:
query = "select s.Student_ID, s.Student_Faculty, s.Student_Faculty_Grade,s.Student_Gender,s.Student_ITI_Status, s.Student_Marital_Status, sum(sea.student_grade) as [Total grade] , c.Company_Name, sc.Salary from Student as s join Student_Company as sc on s.Student_ID = sc.Student_ID join Company as c on sc.Company_ID = c.Company_ID inner join Student_Exam_Answer as sea on sea.Student_ID = s.Student_ID where sea.Exam_ID in (select Exam_ID from exam where Exam_Type = 'Normal') group by s.Student_ID, s.Student_Faculty, s.Student_Faculty_Grade, s.Student_Gender, s.Student_ITI_Status, s.Student_Marital_Status, c.Company_Name, sc.Salary"
df = pd.read_sql(query, conn)

C:\Users\PC\AppData\Local\Temp\ipykernel_27872\3791902152.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [211]:
df.head()

,student_id,student_faculty,student_faculty_grade,student_gender,student_iti_status,student_marital_status,total grade,company_name,salary
0,40151,Faculty of Computers Sciences,Very Good,Female,Graduated,Single,82.0,Etisalat by e&,6505.47
1,14280,Faculty of Computers Sciences,Pass,Female,Graduated,Single,74.0,Orascom Construction,12971.80
2,24431,Faculty of Engineering,Very Good,Male,Graduated,Single,100.0,Talabat (Delivery Hero),14213.00
3,34006,Faculty of Engineering,Very Good,Male,Graduated,Single,86.0,Vodafone Intelligent Solutions (VOIS),5691.00
4,42013,Faculty of Business Administration,Excellent,Male,Graduated,Single,108.0,Vodafone Intelligent Solutions (VOIS),6055.35


In [212]:
df.shape

(3949, 9)

In [213]:
df.isnull().sum()

student_id                0
student_faculty           0
student_faculty_grade     0
student_gender            0
student_iti_status        0
student_marital_status    0
total grade               0
company_name              0
salary                    0
dtype: int64

In [214]:
df.duplicated().sum() 

np.int64(0)

In [215]:
df.drop(columns=['student_id'], inplace=True)
df.head()

,student_faculty,student_faculty_grade,student_gender,student_iti_status,student_marital_status,total grade,company_name,salary
0,Faculty of Computers Sciences,Very Good,Female,Graduated,Single,82.0,Etisalat by e&,6505.47
1,Faculty of Computers Sciences,Pass,Female,Graduated,Single,74.0,Orascom Construction,12971.80
2,Faculty of Engineering,Very Good,Male,Graduated,Single,100.0,Talabat (Delivery Hero),14213.00
3,Faculty of Engineering,Very Good,Male,Graduated,Single,86.0,Vodafone Intelligent Solutions (VOIS),5691.00
4,Faculty of Business Administration,Excellent,Male,Graduated,Single,108.0,Vodafone Intelligent Solutions (VOIS),6055.35


In [216]:
df['student_faculty_grade'] = df['student_faculty_grade'].map({'Pass': 0, 'Good': 1, 'Very Good': 2, 'Excellent': 3})

In [217]:
df['student_iti_status'] = df['student_iti_status'].map({'Failed to Graduate': 0, 'Graduated': 1})

In [218]:
df.head()

,student_faculty,student_faculty_grade,student_gender,student_iti_status,student_marital_status,total grade,company_name,salary
0,Faculty of Computers Sciences,2,Female,1,Single,82.0,Etisalat by e&,6505.47
1,Faculty of Computers Sciences,0,Female,1,Single,74.0,Orascom Construction,12971.80
2,Faculty of Engineering,2,Male,1,Single,100.0,Talabat (Delivery Hero),14213.00
3,Faculty of Engineering,2,Male,1,Single,86.0,Vodafone Intelligent Solutions (VOIS),5691.00
4,Faculty of Business Administration,3,Male,1,Single,108.0,Vodafone Intelligent Solutions (VOIS),6055.35


In [219]:
df = pd.get_dummies(df, columns=["student_faculty","student_gender","student_marital_status"], drop_first=True, dtype= 'uint8')
df.head()

,student_faculty_grade,student_iti_status,total grade,company_name,salary,student_faculty_Faculty of Applied Arts,student_faculty_Faculty of Arts,student_faculty_Faculty of Business Administration,student_faculty_Faculty of Commerce,student_faculty_Faculty of Computers Sciences,student_faculty_Faculty of Economics and Political Science,student_faculty_Faculty of Education,student_faculty_Faculty of Engineering,student_faculty_Faculty of Fine Arts,student_faculty_Faculty of Information Systems,student_faculty_Faculty of Science,student_gender_Male,student_marital_status_Single
0,2,1,82.0,Etisalat by e&,6505.47,0,0,0,0,1,0,0,0,0,0,0,0,1
1,0,1,74.0,Orascom Construction,12971.80,0,0,0,0,1,0,0,0,0,0,0,0,1
2,2,1,100.0,Talabat (Delivery Hero),14213.00,0,0,0,0,0,0,0,1,0,0,0,1,1
3,2,1,86.0,Vodafone Intelligent Solutions (VOIS),5691.00,0,0,0,0,0,0,0,1,0,0,0,1,1
4,3,1,108.0,Vodafone Intelligent Solutions (VOIS),6055.35,0,0,1,0,0,0,0,0,0,0,0,1,1


In [220]:
label_encoder = LabelEncoder()
a = label_encoder.fit_transform(df['company_name'])

In [221]:
df['company_name'] = a
df.head()

,student_faculty_grade,student_iti_status,total grade,company_name,salary,student_faculty_Faculty of Applied Arts,student_faculty_Faculty of Arts,student_faculty_Faculty of Business Administration,student_faculty_Faculty of Commerce,student_faculty_Faculty of Computers Sciences,student_faculty_Faculty of Economics and Political Science,student_faculty_Faculty of Education,student_faculty_Faculty of Engineering,student_faculty_Faculty of Fine Arts,student_faculty_Faculty of Information Systems,student_faculty_Faculty of Science,student_gender_Male,student_marital_status_Single
0,2,1,82.0,16,6505.47,0,0,0,0,1,0,0,0,0,0,0,0,1
1,0,1,74.0,37,12971.80,0,0,0,0,1,0,0,0,0,0,0,0,1
2,2,1,100.0,48,14213.00,0,0,0,0,0,0,0,1,0,0,0,1,1
3,2,1,86.0,52,5691.00,0,0,0,0,0,0,0,1,0,0,0,1,1
4,3,1,108.0,52,6055.35,0,0,1,0,0,0,0,0,0,0,0,1,1


In [222]:
X = df.drop(['salary'], axis=1) 
y = df['salary']

In [223]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [224]:
X_train.dtypes

student_faculty_grade                                           int64
student_iti_status                                              int64
total grade                                                   float64
company_name                                                    int64
student_faculty_Faculty of Applied Arts                         uint8
student_faculty_Faculty of Arts                                 uint8
student_faculty_Faculty of Business Administration              uint8
student_faculty_Faculty of Commerce                             uint8
student_faculty_Faculty of Computers Sciences                   uint8
student_faculty_Faculty of Economics and Political Science      uint8
student_faculty_Faculty of Education                            uint8
student_faculty_Faculty of Engineering                          uint8
student_faculty_Faculty of Fine Arts                            uint8
student_faculty_Faculty of Information Systems                  uint8
student_faculty_Facu

In [225]:
print(X_train.columns.tolist())

['student_faculty_grade', 'student_iti_status', 'total grade', 'company_name', 'student_faculty_Faculty of Applied Arts', 'student_faculty_Faculty of Arts', 'student_faculty_Faculty of Business Administration', 'student_faculty_Faculty of Commerce', 'student_faculty_Faculty of Computers Sciences', 'student_faculty_Faculty of Economics and Political Science', 'student_faculty_Faculty of Education', 'student_faculty_Faculty of Engineering', 'student_faculty_Faculty of Fine Arts', 'student_faculty_Faculty of Information Systems', 'student_faculty_Faculty of Science', 'student_gender_Male', 'student_marital_status_Single']


In [226]:
cat_features = ['student_faculty_Faculty of Applied Arts', 'student_faculty_Faculty of Arts', 'student_faculty_Faculty of Business Administration', 'student_faculty_Faculty of Commerce', 'student_faculty_Faculty of Computers Sciences', 'student_faculty_Faculty of Economics and Political Science', 'student_faculty_Faculty of Education', 'student_faculty_Faculty of Engineering', 'student_faculty_Faculty of Fine Arts', 'student_faculty_Faculty of Information Systems', 'student_faculty_Faculty of Science', 'student_gender_Male', 'student_marital_status_Single']
num_features = ['student_faculty_grade','student_iti_status','company_name','total grade']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', 'passthrough', cat_features)
    ]
)

X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)

In [227]:
class_percentage = df['salary'].value_counts(normalize=True) * 100
print(class_percentage)

salary
12704.00    0.075969
7047.00     0.075969
10999.00    0.050646
7776.00     0.050646
9679.00     0.050646
              ...   
10653.60    0.025323
5127.00     0.025323
9000.00     0.025323
5126.96     0.025323
4508.00     0.025323
Name: proportion, Length: 3851, dtype: float64


In [228]:
y_train_log = np.log1p(y_train)  # log(1 + price) to handle zero values if any
y_test_log = np.log1p(y_test)

In [230]:
from sklearn.metrics import mean_squared_error, r2_score

lin_regS = LinearRegression()
lin_regS.fit(X_train_scaled, y_train_log)

# Predict and evaluate
y_pred_linS = lin_regS.predict(X_test_scaled)

print("RMSE:", np.sqrt(mean_squared_error(y_test_log, y_pred_linS)))
print("R² Score:", r2_score(y_test_log, y_pred_linS))

RMSE: 0.4834172325185249
R² Score: -0.0063159449199226


In [232]:
poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train_scaled)
X_test_poly = poly_features.transform(X_test_scaled)

In [233]:
poly_reg = LinearRegression()
poly_reg.fit(X_train_poly, y_train_log)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [235]:
y_pred_poly = poly_reg.predict(X_test_poly)
mse_poly = mean_squared_error(y_test_log, y_pred_poly)
print(f"Polynomial Regression MSE: {mse_poly}")
print("R² Score:", r2_score(y_test_log, y_pred_poly))

Polynomial Regression MSE: 0.23862218663879592
R² Score: -0.02754516393930717


In [236]:
from sklearn.svm import SVR

# Train SVM model
svm_reg = SVR(kernel='linear')
svm_reg.fit(X_train_scaled, y_train_log)

,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,tol,0.001
,C,1.0
,epsilon,0.1
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [237]:
 
# Predict and evaluate
y_pred_svm = svm_reg.predict(X_test_scaled)
mse_svm = mean_squared_error(y_test_log, y_pred_svm)
print(f"SVM Regression MSE: {mse_svm}")
print("R² Score:", r2_score(y_test_log, y_pred_svm))

SVM Regression MSE: 0.24491466937662362
R² Score: -0.05464159741644137


In [241]:
from sklearn.neighbors import KNeighborsRegressor

# Create k-NN regressor object
knn_reg = KNeighborsRegressor(n_neighbors=5)

# Train the model
knn_reg.fit(X_train_scaled, y_train_log)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [242]:
knn_pred = knn_reg.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test_log, knn_pred)
print(f'Mean Squared Error: {mse}')
print("R² Score:", r2_score(y_test_log, knn_pred))

Mean Squared Error: 0.2820996934506354
R² Score: -0.214766237109125


In [243]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.1)  # alpha is the regularization strength
lasso_reg.fit(X_train_scaled, y_train_log)

,alpha,0.1
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [244]:
y_pred = lasso_reg.predict(X_test_scaled)
mse = mean_squared_error(y_test_log, y_pred)
print(f"LASSO Regression MSE: {mse}")

print("RMSE:", np.sqrt(mean_squared_error(y_test_log, y_pred)))
print("R² Score:", r2_score(y_test_log, y_pred))

LASSO Regression MSE: 0.23294055211738396
RMSE: 0.48263915311274114
R² Score: -0.0030791402305196947


In [245]:
import xgboost as xgb

# Instantiate an XGBoost regressor object
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

# Fit the regressor to the training set
xg_reg.fit(X_train_scaled, y_train_log)

# Predict on test set
preds = xg_reg.predict(X_test_scaled)

# Compute the rmse
rmse = np.sqrt(mean_squared_error(y_test_log, preds))
print("RMSE: %f" % (rmse))
print("R² Score:", r2_score(y_test_log, preds))

RMSE: 0.479444
R² Score: 0.010158457773917329


In [246]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Define the stack of models
estimators = [
    ('random_forest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('gradient_boost', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42))
]

# Create the stacking regressor
stacking_regressor = StackingRegressor(estimators=estimators, final_estimator=LinearRegression(), cv=5)
stacking_regressor.fit(X_train_scaled, y_train_log)

# Predict and evaluate the stacking model
stacked_predictions = stacking_regressor.predict(X_test_scaled)
stacked_mse = mean_squared_error(y_test_log, stacked_predictions)
print(f'Stacked Model MSE: {stacked_mse}')
print("R² Score:", r2_score(y_test_log, stacked_predictions))

Stacked Model MSE: 0.2192288991222766
R² Score: 0.05596542273823568


In [247]:
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

ridge_reg = Ridge(alpha=0.1)
ridge_reg.fit(X_train_scaled, y_train_log)
# Predict and evaluate
y_predRidge = ridge_reg.predict(X_test_scaled)

# Calculate RMSE
rmse = root_mean_squared_error(y_test_log, y_predRidge)

# Calculate MAE
mae = mean_absolute_error(y_test_log, y_predRidge)

# Calculate R-squared
r_squared = r2_score(y_test_log, y_predRidge)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r_squared}")


RMSE: 0.4834154207039306
MAE: 0.39854762111189623
R-squared: -0.006308401727930013
